![R-Kenntnis](../Pics/header.png "R-Kenntnis")

<div class="alert alert-block alert-info">

* **Titel:** Übungen 04 (Add-On): Pivot-Tabellen mit `dplyr`
* **Autor:** Prof. Dr. Denis Royer
* **Datum:** 15.11.2022 (Version 1.2)

</div>

# BI - R-Kenntnis Lösung zu den Übungen 04 (Add-On): Pivot-Tabellen mit `dplyr`

![R-Kenntnis](../Pics/cube.png "Data Cube")

## Intro

Die Firma GroundBreaking Systems ist ein weltweit tätiges IT-Unternehmen, das unterschiedliche IT-Services wie bspw. Server-Hosting, Betreuung von Endgeräten, Beratung, etc. anbietet. Zu den Kunden von GroundBreaking zählen viele internationale Konzerne, aber auch Mittelständler und kleine lokale Firmen oder Organisationen. GroundLake nutzt derzeit ein rudimentäres CRM-System, das einfache Funktionen wie Kundenstammdatenverwaltung oder Erfassung und Pflege von Opportunities anbietet. Um die zukünftige Umsatzentwicklung besser abschätzen zu können, möchte GroundBreaking einen Pipeline-Report erstellen. Das aktuelle CRM-System ist dazu nicht in der Lage und daher werden Sie als Experte für betriebliche Anwendungen engagiert, einen Prototyp für einen Pipeline-Report in Microsoft Excel zu erstellen. Wenn der fertige Prototyp vom GroundBreaking-Management als sinnvoll eingeschätzt wird, soll ein Team mit professionellen Programmierern das bisherige CRM-System erweitern.

<div class="alert alert-block alert-info">

**(Business) Opportunity:** Aus der Sicht des Vertriebs ist eine Opportunity eine Verkaufsmöglichkeit von Produkten oder Services, die durch einen Käufer getätigt werden können. Ein CRM System erlaubt hierbei, den Prozess einer Opportunity von Anlage bis hin zum Kauf adäquat zu unterstützen.
</div>

Um Ihnen die Arbeit zu erleichtern, hat GroundBreaking in den letzten 3 Monaten jeweils am letzten Tag des Monats einen Export aller Opportunities vorgenommen und alle gesammelten Daten in einer Excel-Datei gespeichert. Diese Datei liegt Ihnen vor. Der Grund dafür, dass man insgesamt drei Datenexporte durchgeführt hat, ist, dass man auch die Entwicklung der Pipeline über die Zeit nachverfolgen will. Der Aufbau der Excel-Datei ist in der nachfolgenden Tabelle beschrieben (Data Dictionary).

## Data Dictionary

| Attribut                                                                   | Wert
| -------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| **Export** | 0: Datenexport aller Opportunities vom letzten Tag des  letzten Monats <br>-1: Datenexport vom letzten Tag des vorletzten Monats <br> -2: Datenexport vom letzten Tag des drittletzten Monats
| **Region** | Eine Region, z.B. „Europa“, „Amerika“, etc.                                                   
| **Land** | Ein Land, z.B. „Deutschland“, „Frankreich“, etc.
| **Jahr** | Das Jahr, in dem die Opportunity voraussichtlich geschlossen wird bzw. es voraussichtlich zu einem Geschäftsabschluss kommt.                                                                                                
| **Quartal** | Das Quartal, in dem die Opportunity voraussichtlich geschlossen wird. Quartal bezieht sich immer auf das Jahr, das in der Spalte „Jahr“ angegeben ist.
| **Branche** | Eine Branche, z.B. „Automobil“, „Verbrauchsgüter“, etc.
| **Produkt** | Das Produkt, das im Rahmen der Opportunity angeboten wird <br> Bei der Firma Untersee Marketing wird in einer Opportunity immer nur ein einziges Produkt verwaltet.
| **Phase** | Die Phase, in der sich die Opportunity aktuell befindet. <br>1 – Interessiert (5%) <br>2 – Geprüft (10%) <br>3 – Angebotsvorbereitung (20%) <br>4 – Angebot abgegeben (55%) <br>5 – Angebotsverhandlung (50%) <br>6 – Vertragsverhandlung (75%)
| **Betrag** | Das Volumen (in tausend Euro) der Opportunity bzw. der erwartete Umsatzerlös
| **Anzahl** | <p>Im Datenexport werden in einer Zeile alle Opportunities zusammengefasst, die mit gleichen Rahmenbedingungen erstellt wurden. So können bspw. in einem Quartal eines Jahres mehrere Opportunities für den gleichen Kunden im gleichen Land, für die gleiche Branche, mit dem gleichen Produkt und in der gleichen Phase vorhanden sein. </p>
| **Opportunities** | Die Anzahl Opportunities, die im Datenexport zu einer Zeile zusammengefasst wurden, wird in dieser Spalte angegeben.                                                                                                     
| **Kunde** | Name des Kunden, für den die Opportunity angelegt wurde.                                                                                                                                                             

## Vorbereitungen in R

<div class="alert alert-block alert-warning">
<b>Wichtig:</b> Bevor wir loslegen, müssen wir zunächst einmal ein paar vorbereitende Dinge erledigen:

* Die notwendigen Packages laden (bspw. `tidyverse`)
* Den Datensatz laden (siehe *Data/cube_rohdaten.xlsx*)
* Hinweis: Bei dem Datensartz handelt es sich um den gleich Datensatz, der auch bei der Excel-Übung zum Einsatz kam.
</div>


In [ ]:
library(tidyverse)
library(readxl)
library(DT)

# Hinweise der summarise() Funktion bei der Gruppierung von
# Daten abschaltem - mehr unter:
# https://statisticsglobe.com/dplyr-message-summarise-has-grouped-output-r
options(dplyr.summarise.inform = FALSE)

In [ ]:
# Daten laden 
pipeline_report.raw <- read_excel("../Data/cube_rohdaten.xlsx")

In [ ]:
# Die ersten 10 Zeilen ausgeben
DT::datatable(head(pipeline_report.raw,n = 10))

In [ ]:
# Wie sieht der Datensatz aus?
str(pipeline_report.raw)
summary(pipeline_report.raw)

# Einfache Pivots erstellen mit `dplyr` (Aufgabe 2)

In den Folgenden Source-Code Blöcken, werden die Aufgaben aus der Excel Pivot-Tabellen Übung in ***R*** nachgestellt. 

Die Aufgaben sollen als Beispiele für die Nutzung von `dplyr` dienen und die Inhalte aus **Übung 04:** <a href="../Uebung 04/R-Kenntnis Uebung 04.ipynb">Datenaufbereitung mit `dplyr`</a> ergänzen.

## Aufgabe 2a

**Frage:**  Wie viele Opportunities, bei denen man sich aktuell in Vertragsverhandlungen befindet, werden voraussichtlich im 2. Quartal 2019 abgeschlossen?


### SCHRITT 1: Export und QuartalJahr
 Zunächst geht es darum, die Daten einmal zu filtern, so dass nur die Daten mit `Export = 0` gentzt werden.

Weiterhin werden die Daten zu den Quartale und Jahre so kombiniert,
so dass sie in der Form **`YYYY-Q`** in eine neue Spalte QuartalJahr
zum Gruppieren der Daten genutzt werden können. 

Das ganze in Pseudo-Code Schreibweise:

```
Nimm die Tabelle „pipeline_report.raw “ UND DANN
    filtere alle Daten aus dem aktuellen Export raus UND DANN
    erweitere die Tabelle um eine Spalte QuartalJahr mit YYYY-Q als Inhalt
```

In [ ]:
aufgabe.2a.step1 <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) 

head(aufgabe.2a.step1)

### SCHRITT 2: Gruppieren und Summieren
Sind die Daten nun aufbereitet, können sie nach QuartalJahr und Phase
gruppiert (group_by) und die Anzahl der Opportunities dann addiert werden
(summarise). Bei der Addition werden die leeren Werte (NA) ignoriert
(Parameter: na.rm = TRUE).

```
Nimm die Tabelle „aufgabe.2a.step1 “ UND DANN
    gruppiere die Daten nach den Spalten QuartalJahr und Phase  UND DANN
    summiere die Daten entlang der oben durchgeführten Gruppierung für die 
    Spalte AnzahlOpportunities durch und ignoriere die leeren Felder
```

In [ ]:
aufgabe.2a.step2 <- aufgabe.2a.step1 %>%
              group_by(QuartalJahr, Phase) %>%
              summarise(Auftraege = sum(AnzahlOpportunities,na.rm = TRUE)) 

# Ausagabe der ersten 6 Werte
head(aufgabe.2a.step2)

### SCHRITT 3: Tabelle pivotieren
Da die Daten immer noch in ihrer Ursprungsform sind (zeilenweise), 
müssen sie nun einmal pivotiert werden, so dass die Phasen der Projekte
und die Qartalssummen gebildet werden können. 
Die pivot_wider() Funktion übernimmt dies und sortiert die Daten so um, dass in 
den Zeilen die Projektphasen stehen (values_from = Auftraege) und in den Splaten die
QuartalJahr Informationen (names_from = QuartalJahr).


```
Nimm die Tabelle „aufgabe.2a.step2 “ UND DANN
    pivotiere die Daten so um, dass die Status in den Zeilen und die
    Quartale (QuartalJahr) in den Spalten stehen. Die Daten aus der Spalte Auftraege
    werden dann jeweils in die neu entstehende Tabelle einsortiert
```

In [ ]:
aufgabe.2a.step3 <- aufgabe.2a.step2 %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege)

aufgabe.2a.step3

### SCHRITT 4: Zeilensummen
Im vorletzten Schritt, sollen die Summen der Zeilen am Ende als neue Spalte eimngefügt werden. Hierzu gibt es zwei Wege:

**Weg 1:**
Die Splaten, die summiert werden sollen, werden vorab in einem separaten Vektor gespeichert (cols_to_sum) und dann summiert.

Mehr unter <https://data-se.netlify.app/2021/05/27/zeilenweise-operationen-tidyverse-stil/>

In [ ]:
cols_to_sum <- c("2019-1", "2019-2", "2019-3", "2019-4", "2020-1")
aufgabe.2a.step4 <- aufgabe.2a.step3 %>%
        rowwise() %>%              
        mutate(summe = sum(across(any_of(cols_to_sum)),na.rm = TRUE))

In [ ]:
aufgabe.2a.step4

**Weg 2:**
Alternativ kann man einfach alle Spalten summieren, die Nummern enthalten - dies geht dann wie folgt:

In [ ]:
aufgabe.2a.step4 <- aufgabe.2a.step3 %>%
        rowwise() %>%
        mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))

In [ ]:
aufgabe.2a.step4

### SCHRITT 5: Spaltensummen

Im letzten Schritt, sollen die Summen der Spalten am Ende als neue Zeile eimngefügt werden. Hierzu gibt es im Package `janitor` die funktion `adorn_totals()`. Mehr zu `janitor` finden sie hier: 
 * <https://cran.r-project.org/web/packages/janitor/vignettes/janitor.html>
 * <https://github.com/sfirke/janitor>
 * <https://sfirke.github.io/janitor/articles/tabyls.html>

In [ ]:
library(janitor)
aufgabe.2a.step5 <- aufgabe.2a.step4 %>%
  adorn_totals("row")

In [ ]:
aufgabe.2a.step5

### Alle Schritte zusammen

In [ ]:
aufgabe.2a <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) %>%
              group_by(QuartalJahr, Phase) %>%
              summarise(Auftraege = sum(AnzahlOpportunities)) %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege) %>%
              #rowwise() %>%              
              #mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE)) %>%
              adorn_totals("col") %>%
              adorn_totals("row")

In [ ]:
aufgabe.2a

## Aufgabe 2b

**Frage:** Wie viele Opportunities werden voraussichtlich im 1. Quartal 2019 in Deutschland abgeschlossen?

In [ ]:
aufgabe.2b <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) %>%
              group_by(QuartalJahr, Land) %>%
              summarise(Auftraege = sum(AnzahlOpportunities)) %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE)) %>%
              adorn_totals("row")

aufgabe.2b

## Aufgabe 2c

**Frage:** Für welches Produkt existieren derzeit in den USA die meisten Opportunities?

In [ ]:
aufgabe.2c <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              group_by(Land, Produkt) %>%
              summarise(Auftraege = sum(AnzahlOpportunities)) %>%
              pivot_wider(names_from = Land, values_from = Auftraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE)) %>%
              adorn_totals("row")

aufgabe.2c

## Aufgabe 2d

**Frage:** Mit welchem Produkt wird in welcher Branche voraussichtlich am meisten und am wenigsten Umsatz erzielt werden?

In [ ]:
aufgabe.2d <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              group_by(Branche, Produkt) %>%
              summarise(SummeBetraege = sum(Betrag)) %>%
              pivot_wider(names_from = Branche, values_from = SummeBetraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE)) %>%
              adorn_totals("row")

DT::datatable(aufgabe.2d)

## Exkurs: Tabellen aufhübschen mit gt

In [ ]:
library(gt)

# Tabelle mit dem Package gt hübsch machen
myTable <- aufgabe.2d %>%
  gt() %>%
  tab_header(
    title = "Umsatz Dienstleistungen nach Länder",
  ) %>%
  fmt_currency(
    columns = c(2:11),
    currency = "EUR",
  ) 

# Ausgabe als HTML - ist notwendig, damit es in JupyterLab
# auch richtig eingebettet wird
gt:::as.tags.gt_tbl(myTable)

# Kompexere Pivots (Aufgabe 3)

In [ ]:
# Weitere Infos und Beispiele zur case_when() Funktion
# https://statologie.de/neue-variablen-mutate-case_when-r/

aufgabe.3a <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) %>%
              mutate(propabitity = case_when(
                  Phase == '1 Interessiert' ~ 0.05,
                  Phase == '2 Geprüft' ~ 0.10,
                  Phase == '3 Angebotsvorbereitung' ~ 0.20,
                  Phase == '4 Angebot abgegeben' ~ 0.25,
                  Phase == '5 Angebotsverhandlung' ~ 0.50,
                  Phase == '6 Vertragsverhandlung' ~ 0.75,
                  TRUE ~ as.numeric(0))) %>%                # Wenn nichts passt 0 zurückgeben!
              mutate(GewBetrag = propabitity * Betrag) %>%
              group_by(QuartalJahr, Produkt) %>%
              summarise(Auftraege = sum(GewBetrag,na.rm = TRUE)) %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE)) %>%
              adorn_totals("row")

In [ ]:
DT::datatable(aufgabe.3a)